# 导入相关软件包

In [1]:
import os
import random

import pandas as pd
import numpy as np

import mindspore as ms
from mindspore import context
from mindspore.common import set_seed

# 参数配置

In [2]:
def check_device_availability():
    if context.get_context(attr_key='device_target') == "GPU":
        device_target = "GPU"
        context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
    else:
        device_target = "CPU"
        context.set_context(mode=context.GRAPH_MODE, device_target="CPU")
    return device_target

In [3]:
check_device_availability()

'CPU'

In [4]:
# 随机种子
def seed_mindspore(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [5]:
config_para = {
    'seed':42,
}

In [6]:
seed_mindspore(seed=config_para['seed'])

# 数据读入

In [7]:
# 西瓜数据集3.0α
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
    'density': [0.697, 0.774, 0.634, 0.608, 0.556, 0.403, 0.481, 0.437, 0.666, 0.243, 0.245, 0.343, 0.639, 0.657, 0.360, 0.593, 0.719],
    'sugar_rate': [0.460, 0.376, 0.264, 0.318, 0.215, 0.237, 0.149, 0.211, 0.091, 0.267, 0.057, 0.099, 0.161, 0.198, 0.370, 0.042, 0.103],
    'is_good': ['是', '是', '是', '是', '是', '否', '否', '否', '否', '否', '否', '否', '否', '否', '否', '否', '否']
}

In [8]:
# 创建DataFrame
df = pd.DataFrame(data)
df

,id,density,sugar_rate,is_good
0,1,0.697,0.460,是
1,2,0.774,0.376,是
2,3,0.634,0.264,是
3,4,0.608,0.318,是
4,5,0.556,0.215,是
5,6,0.403,0.237,否
6,7,0.481,0.149,否
7,8,0.437,0.211,否
8,9,0.666,0.091,否
9,10,0.243,0.267,否


In [9]:
# 将中文类别映射为数字label
mapping = { # 映射字典
    'is_good': {'否': 0, '是': 1}
}

# 编码
for column, map_dict in mapping.items():
    df[column] = df[column].map(map_dict)
    
# 去除 "编号" 列
df = df.drop(['id'], axis=1)

In [10]:
df

,density,sugar_rate,is_good
0,0.697,0.460,1
1,0.774,0.376,1
2,0.634,0.264,1
3,0.608,0.318,1
4,0.556,0.215,1
5,0.403,0.237,0
6,0.481,0.149,0
7,0.437,0.211,0
8,0.666,0.091,0
9,0.243,0.267,0


# 数据加载和处理

In [11]:
from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split

In [12]:
# 分离特征和标签

features = df.drop('is_good', axis=1).to_numpy()
labels = df['is_good'].to_numpy()

In [13]:
features.shape, labels.shape

((17, 2), (17,))

In [14]:
# 特征归一化
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [15]:
# 此时数据仅供训练，不再分割测试集了

# 可视化和探索性数据分析

# SVM实现：libsvm

In [16]:
from libsvm.svmutil import *

In [17]:
x_train = features.tolist() if isinstance(features, np.ndarray) else features
y_train = labels.tolist() if isinstance(labels, np.ndarray) else labels

In [19]:
# 建立模型
prob = svm_problem(y_train, x_train)

In [20]:
# 1. 线性核
param_1 = svm_parameter('-t 0 -c 1')
svm_linear_1 = svm_train(prob, param_1)

In [21]:
# 2. 高斯核
param_2 = svm_parameter('-t 2 -c 1 -g 0.5')
svm_rbf_1 = svm_train(prob, param_2)

In [22]:
# 提取支持向量并对比
linear_support_vectors_1 = svm_linear_1.get_SV()
rbf_support_vectors_1 = svm_rbf_1.get_SV()

print(f"libsvm 线性核：{linear_support_vectors_1}")
print(f"libsvm 高斯核：{rbf_support_vectors_1}")

libsvm 线性核：[{1: 1.4905482969647703, 2: 1.4030719888143706}, {1: 0.6259358312625644, 2: 0.4400406021155382}, {1: 0.46536494477501167, 2: 0.9043593064167609}, {1: 0.14422317179990707, 2: 0.018714370434798854}, {1: 0.656814847894786, 2: -0.4456043338664239}, {1: 0.7679793077707839, 2: -0.12746003647484525}, {1: -1.0662342801831814, 2: 1.3514810216697901}, {1: 1.1508791140103318, 2: -0.9443170162640336}]
libsvm 高斯核：[{1: 1.0150114408285567, 2: 2.125345528838495}, {1: 1.4905482969647703, 2: 1.4030719888143706}, {1: 0.6259358312625644, 2: 0.4400406021155382}, {1: 0.46536494477501167, 2: 0.9043593064167609}, {1: 0.14422317179990707, 2: 0.018714370434798854}, {1: -0.5906974240469682, 2: -0.01567960766158805}, {1: -1.776451662724279, 2: -1.3398477643724827}, {1: 0.656814847894786, 2: -0.4456043338664239}, {1: 0.7679793077707839, 2: -0.12746003647484525}, {1: -1.0662342801831814, 2: 1.3514810216697901}]


# SVM实现：sklearn

In [23]:
from sklearn.svm import SVC

In [24]:
# 1. 线性核
svm_linear_2 = SVC(kernel='linear')
svm_linear_2.fit(features, labels)

SVC(kernel='linear')

In [25]:
# 2. 高斯核
svm_rbf_2 = SVC(kernel='rbf', gamma='scale')
svm_rbf_2.fit(features, labels)

SVC()

In [26]:
# 提取支持向量并对比
linear_support_vectors_2 = svm_linear_2.support_vectors_
rbf_support_vectors_2 = svm_rbf_2.support_vectors_

print(f"sklearn 线性核：{linear_support_vectors_2}")
print(f"sklearn 高斯核：{rbf_support_vectors_2}")

sklearn 线性核：[[ 0.65681485 -0.44560433]
 [ 0.76797931 -0.12746004]
 [-1.06623428  1.35148102]
 [ 1.15087911 -0.94431702]
 [ 1.4905483   1.40307199]
 [ 0.62593583  0.4400406 ]
 [ 0.46536494  0.90435931]
 [ 0.14422317  0.01871437]]
sklearn 高斯核：[[-0.59069742 -0.01567961]
 [-1.77645166 -1.33984776]
 [ 0.65681485 -0.44560433]
 [ 0.76797931 -0.12746004]
 [-1.06623428  1.35148102]
 [ 1.01501144  2.12534553]
 [ 1.4905483   1.40307199]
 [ 0.62593583  0.4400406 ]
 [ 0.46536494  0.90435931]
 [ 0.14422317  0.01871437]]


# SVM实现：MindSpore

In [27]:
from mindspore import Tensor, Parameter, nn, ops

In [28]:
# 设置运行模式（动态图适合调试）
context.set_context(mode=context.PYNATIVE_MODE)

In [29]:
# 因为mindspore里没有svm，这里自定义了一个，并使用mindspore的tensor
class SVM(nn.Cell):
    def __init__(self, n_features, kernel='linear', gamma=1.0):
        super(SVM, self).__init__()
        self.kernel = kernel
        self.gamma = gamma
        self.w = Parameter(Tensor(np.random.randn(n_features, 1).astype(np.float32)), name="w")  # 随机初始化
        self.b = Parameter(Tensor(np.random.randn(1).astype(np.float32)), name="b")  # 随机初始化

    def kernel_function(self, X, Y):
        if self.kernel == 'linear':
            return ops.MatMul()(X, Y.T)
        elif self.kernel == 'rbf':
            X_norm = ops.ReduceSum(keep_dims=True)(ops.Pow()(X, 2), axis=1)
            Y_norm = ops.ReduceSum(keep_dims=True)(ops.Pow()(Y, 2), axis=1)
            XY = ops.MatMul()(X, Y.T)
            return ops.Exp()(-self.gamma * (X_norm - 2 * XY + Y_norm.T))
        else:
            raise ValueError("Unsupported kernel type")

    def construct(self, X):
        return ops.MatMul()(X, self.w) + self.b
    
# Convert data to MindSpore tensors
X_train_tensor = Tensor(features, dtype=ms.float32)
y_train_tensor = Tensor(np.where(labels == 0, -1, 1)[:, None], dtype=ms.float32)

In [30]:
# 1. 线性核
svm_linear_3 = SVM(n_features=2, kernel='linear')
optimizer_linear = nn.SGD(params=svm_linear_3.trainable_params(), learning_rate=0.01)
loss_fn_linear = nn.WithLossCell(svm_linear_3,loss_fn=nn.MSELoss(reduction='mean'))  # 使用 Mean 操作，将 MSELoss 输出的张量压缩为标量

# 使用 TrainOneStepCell 将前向传播、反向传播和优化器结合
train_step = nn.TrainOneStepCell(loss_fn_linear, optimizer_linear)
train_step.set_train()

for epoch in range(100):
    loss = train_step(X_train_tensor, y_train_tensor)
    print(f"Epoch {epoch + 1}, Loss: {loss}")
    print(f"Weight w: {svm_linear_3.w.asnumpy().flatten()}, Bias b: {svm_linear_3.b.asnumpy()}")

Epoch 1, Loss: 3.1184194
Weight w: [-1.0820893   0.32838973], Bias b: [0.26522517]
Epoch 2, Loss: 3.0028596
Weight w: [-1.0529315   0.33757082], Bias b: [0.25168538]
Epoch 3, Loss: 2.892303
Weight w: [-1.0243928   0.34645405], Bias b: [0.23841637]
Epoch 4, Loss: 2.786528
Weight w: [-0.9964598   0.35504782], Bias b: [0.22541276]
Epoch 5, Loss: 2.6853254
Weight w: [-0.9691191   0.36336026], Bias b: [0.21266921]
Epoch 6, Loss: 2.5884922
Weight w: [-0.9423577   0.37139934], Bias b: [0.20018053]
Epoch 7, Loss: 2.495837
Weight w: [-0.9161631  0.3791728], Bias b: [0.18794163]
Epoch 8, Loss: 2.4071753
Weight w: [-0.8905228   0.38668817], Bias b: [0.1759475]
Epoch 9, Loss: 2.3223324
Weight w: [-0.86542475  0.39395276], Bias b: [0.16419326]
Epoch 10, Loss: 2.24114
Weight w: [-0.84085715  0.40097374], Bias b: [0.1526741]
Epoch 11, Loss: 2.1634376
Weight w: [-0.8168084   0.40775806], Bias b: [0.14138532]
Epoch 12, Loss: 2.0890727
Weight w: [-0.7932672   0.41431245], Bias b: [0.13032232]
Epoch 13, 

In [31]:
# 2. 高斯核
svm_rbf_3 = SVM(n_features=2, kernel='rbf', gamma=0.5)
optimizer_rbf = nn.SGD(params=svm_rbf_3.trainable_params(), learning_rate=0.01)
loss_fn_rbf = nn.WithLossCell(svm_rbf_3,loss_fn=nn.MSELoss(reduction='mean'))  # 使用 Mean 操作，将 MSELoss 输出的张量压缩为标量

# 使用 TrainOneStepCell 将前向传播、反向传播和优化器结合
train_step = nn.TrainOneStepCell(loss_fn_rbf, optimizer_rbf)
train_step.set_train()

for epoch in range(100):
    loss = train_step(X_train_tensor, y_train_tensor)
    print(f"Epoch {epoch + 1}, Loss: {loss}")
    print(f"Weight w: {svm_rbf_3.w.asnumpy().flatten()}, Bias b: {svm_rbf_3.b.asnumpy()}")

Epoch 1, Loss: 1.7509526
Weight w: [ 1.0013833  -0.56171006], Bias b: [-0.5229017]
Epoch 2, Loss: 1.7057446
Weight w: [ 0.99235886 -0.5428896 ], Bias b: [-0.520679]
Epoch 3, Loss: 1.6620594
Weight w: [ 0.9834412 -0.5244102], Bias b: [-0.5185007]
Epoch 4, Loss: 1.6198448
Weight w: [ 0.9746294 -0.5062654], Bias b: [-0.51636595]
Epoch 5, Loss: 1.5790498
Weight w: [ 0.9659228  -0.48844898], Bias b: [-0.51427394]
Epoch 6, Loss: 1.5396261
Weight w: [ 0.9573205  -0.47095478], Bias b: [-0.5122238]
Epoch 7, Loss: 1.5015265
Weight w: [ 0.9488217  -0.45377678], Bias b: [-0.5102146]
Epoch 8, Loss: 1.4647055
Weight w: [ 0.94042563 -0.43690902], Bias b: [-0.50824565]
Epoch 9, Loss: 1.4291198
Weight w: [ 0.9321314  -0.42034572], Bias b: [-0.506316]
Epoch 10, Loss: 1.3947268
Weight w: [ 0.92393816 -0.40408123], Bias b: [-0.504425]
Epoch 11, Loss: 1.3614858
Weight w: [ 0.91584504 -0.3881099 ], Bias b: [-0.50257176]
Epoch 12, Loss: 1.3293576
Weight w: [ 0.9078512 -0.3724263], Bias b: [-0.5007556]
Epoch 

In [32]:
# 支持向量筛选函数
def find_support_vectors(model, X, y, kernel='linear', gamma=1.0):
    # 转换为 numpy 以便计算
    X_np = X.asnumpy()
    y_np = y.asnumpy()

    if kernel == 'linear':
        # 对于线性核，计算 wx + b
        decision_values = np.dot(X_np, model.w.asnumpy()) + model.b.asnumpy()
    elif kernel == 'rbf':
        # 对于 RBF 核函数，计算核矩阵并加权
        def rbf_kernel(X1, X2, gamma):
            X1_norm = np.sum(np.square(X1), axis=1, keepdims=True)
            X2_norm = np.sum(np.square(X2), axis=1, keepdims=True)
            K = np.exp(-gamma * (X1_norm - 2 * np.dot(X1, X2.T) + X2_norm.T))
            return K
        
        # 直接使用模型的预测结果作为决策值
        decision_values = model(Tensor(X_np, dtype=ms.float32)).asnumpy()

    else:
        raise ValueError("Unsupported kernel type")

    # 筛选支持向量（满足间隔条件 |y * f(x)| <= 1）
    margins = y_np.flatten() * decision_values.flatten()
    support_indices = np.where((margins > 0) & (margins <= 1.0))[0]  # 筛选支持向量
    support_vectors = X_np[support_indices]

    return support_indices, support_vectors


_, linear_support_vectors_3 = find_support_vectors(
    svm_linear_3, X_train_tensor, y_train_tensor, kernel='linear'
)
_, rbf_support_vectors_3 = find_support_vectors(
    svm_rbf_3, X_train_tensor, y_train_tensor, kernel='rbf', gamma=0.5
)

print(f"mindspore 线性核：{linear_support_vectors_3}")
print(f"mindspore 高斯核：{rbf_support_vectors_3}")

mindspore 线性核：[[ 1.4905483   1.403072  ]
 [ 0.62593585  0.4400406 ]
 [ 0.46536493  0.9043593 ]
 [-0.80067474  0.20788126]
 [-0.31896207 -0.5487863 ]
 [-0.5906974  -0.01567961]
 [ 0.82356155 -1.047499  ]
 [-1.7888032   0.46583608]
 [ 0.6568149  -0.44560432]
 [ 0.7679793  -0.12746003]
 [ 1.1508791  -0.94431704]]
mindspore 高斯核：[[ 1.0150114   2.1253455 ]
 [ 1.4905483   1.403072  ]
 [ 0.62593585  0.4400406 ]
 [ 0.46536493  0.9043593 ]
 [-0.80067474  0.20788126]
 [-0.31896207 -0.5487863 ]
 [-0.5906974  -0.01567961]
 [ 0.82356155 -1.047499  ]
 [ 0.6568149  -0.44560432]
 [ 0.7679793  -0.12746003]
 [-1.0662342   1.3514811 ]
 [ 0.3727279  -1.4688252 ]
 [ 1.1508791  -0.94431704]]
